# The Kopi Latte Ratio Project: Data Preparation

The objective of this notebook is to clean up the location & reviews data collected from Google Maps, and prepare a dataset that can be used to train a review text classification model.

## Load Extracted Data

In [3]:
import os
os.chdir('..')

In [4]:
import pandas as pd

places_df = pd.read_csv('data/raw/places.csv', index_col=0)
places_df.head()

,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,...,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,permanently_closed
0,OPERATIONAL,"136 Bedok North Ave 3, #01-152, Singapore 460136",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Percolate,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJV6HD-Eo92jERjhfY7NEDrOM,2.0,4.4,...,1.328262,103.935252,1.329291,103.936559,1.326591,103.933859,True,8WHP+84 Singapore,6PH58WHP+84,NaN
1,OPERATIONAL,"216 Bedok North Street 1, #01-32, Singapore 46...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Generation Coffee Roasters (Bedok),"[{'height': 4000, 'html_attributions': ['<a hr...",ChIJhbwWY-I92jERxtB-gF22sL0,NaN,4.6,...,1.327248,103.933039,1.328730,103.934367,1.326030,103.931668,False,8WGM+V6 Singapore,6PH58WGM+V6,NaN
2,OPERATIONAL,"744 Bedok Reservoir Rd, #01-3029 Reservoir Vil...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Refuel Cafe,"[{'height': 2268, 'html_attributions': ['<a hr...",ChIJcf_SpPk82jERM28p3SYNBnI,2.0,4.2,...,1.337519,103.921323,1.338910,103.922575,1.336211,103.919875,True,8WQC+2G Singapore,6PH58WQC+2G,NaN
3,OPERATIONAL,"537 Bedok North Street 3, #01-575, Singapore 4...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Marie's Lapis Cafe,"[{'height': 3072, 'html_attributions': ['<a hr...",ChIJ3Vc6OY092jERhObI1bZ_4Sk,NaN,4.7,...,1.331827,103.924498,1.333259,103.925757,1.330559,103.923058,True,8WJF+PQ Singapore,6PH58WJF+PQ,NaN
4,OPERATIONAL,"311 New Upper Changi Rd #01-78 Bedok Mall, Sin...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,COFFEESARANG,"[{'height': 526, 'html_attributions': ['<a hre...",ChIJOb_8OAwj2jERPK-QVelr5Vk,NaN,4.1,...,1.325154,103.929854,1.326789,103.931245,1.324089,103.928545,True,8WGH+3W Singapore,6PH58WGH+3W,NaN


## Data Exploration

In [101]:
places_df.columns

Index(['business_status', 'formatted_address', 'icon', 'icon_background_color',
       'icon_mask_base_uri', 'name', 'photos', 'place_id', 'price_level',
       'rating', 'reference', 'types', 'user_ratings_total',
       'geometry.location.lat', 'geometry.location.lng',
       'geometry.viewport.northeast.lat', 'geometry.viewport.northeast.lng',
       'geometry.viewport.southwest.lat', 'geometry.viewport.southwest.lng',
       'opening_hours.open_now', 'plus_code.compound_code',
       'plus_code.global_code', 'permanently_closed'],
      dtype='object')

In [102]:
places_df['place_id'].nunique()

4884

In [103]:
reviews_df['place_id'].nunique()

4692

In [104]:
len(reviews_df)

21685

With 4884 relevant locations and 21685 reviews pulled from Google Maps, I came up with a system to efficiently classify the locations as a cafe or kopitiam. There will be 3 layers of classification in order to minimise the manual effort required:
1. Heuristic classification - Classify the locations based on popular coffee franchises and keywords in names. This requires the least effort and can classify the greatest number of locations accurately. It will also enable
2. Review text classification - Train a model to classify the location based on its reviews. This will automate the classification process.
3. Manual human classification - For locations that are still ambiguous, I will manually classify them. This requires the most effort and should be limited to a reasonable number of location

## Heuristic Classification

In [105]:
# many coffee franchise locations are in the format "<franchise name> - <location>"
# the franchise store names by splitting 
franchise_names = places_df['name'].str.split('-',expand=True)[0].str.split('(', expand=True)[0].rename('franchise_name')

# get top 20 franchise names
franchise_names_top = franchise_names.value_counts().head(20)
franchise_names_top

franchise_name
Toast Box                        54
The Coffee Bean & Tea Leaf       34
Ya Kun Kaya Toast                34
luckin coffee                    25
Kopitiam                         24
Kimly Coffeeshop                 24
Fun Toast                        19
The Coffee Bean and Tea Leaf     14
Tiong Bahru Bakery               14
Killiney Kopitiam                14
S                                13
Huggs Coffee                     12
Happy Hawkers                    11
Kimly Coffeeshop                 10
Kopi & Tarts                     10
Baker & Cook                      9
Han's Cafe                        8
Kopitiam Corner                   8
Koufu                             8
De Tian Coffee House              8
Name: count, dtype: int64

In [106]:
import numpy as np

# Define list of cafe & kopi franchises and classify locations accordingly
cafe_franchises = ['Starbucks', 'The Coffee Bean & Tea Leaf', 'luckin coffee', 'The Coffee Bean and Tea Leaf', 'Tiong Bahru Bakery', 'Huggs Coffee', 'Baker & Cook']
kopi_franchises = ['Ya Kun Kaya Toast', 'Toast Box', 'Kopitiam', 'Kimly Coffeeshop', 'Fun Toast', 'Kopi Kiosk', 'S-11', 'Toast Box', 'Killiney Kopitiam', 'Happy Hawkers', 'Kopi & Tarts', 'De Tian Coffee House'] 

places_df['is_cafe'] = np.where(places_df['name'].str.contains('|'.join(cafe_franchises), regex=True, case=False), 1,0)
places_df['is_kopitiam'] = np.where(places_df['name'].str.contains('|'.join(kopi_franchises), regex=True, case=False), 1,0)
places_df[['is_cafe', 'is_kopitiam']].value_counts()

is_cafe  is_kopitiam
0        0              4268
         1               418
1        0               197
         1                 1
Name: count, dtype: int64

In [107]:
# check location that was labelled as both kopitiam & cafe
places_df.loc[(places_df['is_cafe'] == 1) & (places_df['is_kopitiam'] == 1)]

,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,...,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,permanently_closed,is_cafe,is_kopitiam
9966,OPERATIONAL,"277C Compassvale Link, #01-13 (Shop 2A Kopitia...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Starbucks Kopitiam City,"[{'height': 706, 'html_attributions': ['<a hre...",ChIJ3UHR8T0X2jERdBuJJ2ly4EY,2.0,3.6,...,1.383738,103.895824,1.381038,103.892632,True,9VJV+X9 Singapore,6PH59VJV+X9,NaN,1,1


In [108]:
places_df.loc[places_df['place_id'] == 'ChIJ3UHR8T0X2jERdBuJJ2ly4EY', 'is_kopitiam'] = 0
places_df[['is_cafe', 'is_kopitiam']].value_counts()

is_cafe  is_kopitiam
0        0              4268
         1               418
1        0               198
Name: count, dtype: int64

In [109]:
# Get top 50 words in the place names
pd.Series(' '.join(places_df['name']).lower().split()).value_counts()[:20]

coffee        709
cafe          487
@             340
&             326
the           292
-             272
house         246
food          230
kopitiam      186
shop          149
toast         147
coffeeshop    141
tea           138
eating        125
kopi           99
centre         92
mall           87
restaurant     84
café           84
bar            80
Name: count, dtype: int64

In [110]:
# Check number of samples for each label
kopi_word_list = ['kopitiam', 'toast', 'kopi']
places_df.loc[places_df['name'].str.contains('|'.join(kopi_word_list), regex=True, case=False), 'is_kopitiam'] = 1
places_df[['is_cafe', 'is_kopitiam']].value_counts()

is_cafe  is_kopitiam
0        0              4144
         1               542
1        0               197
         1                 1
Name: count, dtype: int64

After classifying the locations based on coffee franchises & keywords, we are able to quickly identify about 500 kopitiams and 200 cafes. This also gives us a good base to create a training dataset for our review text classification model.

In [113]:
conditions = (places_df['is_cafe'] == 1) & (places_df['is_kopitiam'] == 1)
places_df.loc[conditions]

,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,...,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,permanently_closed,is_cafe,is_kopitiam
9966,OPERATIONAL,"277C Compassvale Link, #01-13 (Shop 2A Kopitia...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Starbucks Kopitiam City,"[{'height': 706, 'html_attributions': ['<a hre...",ChIJ3UHR8T0X2jERdBuJJ2ly4EY,2.0,3.6,...,1.383738,103.895824,1.381038,103.892632,True,9VJV+X9 Singapore,6PH59VJV+X9,NaN,1,1


In [151]:
# starbucks kopitiam location is wrongly classified as both cafe & kopitiam, remove kopitiam label
places_df.loc[places_df['place_id'] == 'ChIJ3UHR8T0X2jERdBuJJ2ly4EY', 'is_kopitiam'] = 0
places_df.loc[conditions]

,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,...,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,permanently_closed,is_cafe,is_kopitiam
9966,OPERATIONAL,"277C Compassvale Link, #01-13 (Shop 2A Kopitia...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Starbucks Kopitiam City,"[{'height': 706, 'html_attributions': ['<a hre...",ChIJ3UHR8T0X2jERdBuJJ2ly4EY,2.0,3.6,...,1.383738,103.895824,1.381038,103.892632,True,9VJV+X9 Singapore,6PH59VJV+X9,NaN,1,0


In [164]:
places_labelled = places_df.loc[places_df[['is_cafe', 'is_kopitiam']].sum(axis=1) > 0] 
places_labelled.head()
# places_labelled.to_csv('data/interim/places_labelled.csv', index=False)

In [163]:
places_unlabelled = places_df.loc[places_df[['is_cafe', 'is_kopitiam']].sum(axis=1) == 0] 
places_unlabelled.head()
# places_unlabelled.to_csv('data/interim/places_unlabelled.csv', index=False)

## Prepare reviews dataset for text classification

In [5]:
# load reviews data and drop potentially identifying columns
reviews_df = pd.read_csv('data/raw/reviews.csv')
reviews_df = reviews_df.drop(columns={'author_name', 'author_url'})
reviews_df.head()

,language,original_language,profile_photo_url,rating,relative_time_description,text,time,translated,place_id
0,en,en,https://lh3.googleusercontent.com/a/ACg8ocK9Ir...,5,2 months ago,It’s great to find a cafe that serves good cof...,1716184446,False,ChIJV6HD-Eo92jERjhfY7NEDrOM
1,en,en,https://lh3.googleusercontent.com/a-/ALV-UjUNd...,5,2 months ago,Nice little independent cafe in Bedok. Quite a...,1715937637,False,ChIJV6HD-Eo92jERjhfY7NEDrOM
2,en,en,https://lh3.googleusercontent.com/a-/ALV-UjWrw...,5,a month ago,Really blessed this is just around the neighbo...,1717770283,False,ChIJV6HD-Eo92jERjhfY7NEDrOM
3,en,en,https://lh3.googleusercontent.com/a/ACg8ocKJ2N...,5,2 months ago,Crowded on a PH morning and I can see why! Cof...,1716353007,False,ChIJV6HD-Eo92jERjhfY7NEDrOM
4,en,en,https://lh3.googleusercontent.com/a-/ALV-UjUBh...,5,2 months ago,This place have very delicious garlic cheese c...,1715824235,False,ChIJV6HD-Eo92jERjhfY7NEDrOM


In [7]:
len(reviews_df)

21685

In [155]:
# check character count of longest review
reviews_df['char_count'] = reviews_df['text'].str.len()
reviews_df['char_count'].max()

4085.0

In [171]:
# Join reviews with places
reviews_train_df = places_labelled.merge(reviews_df, on='place_id', how='inner')[['text', 'place_id', 'name', 'is_cafe', 'is_kopitiam']]

# Convert is_cafe and is_kopitiam to single binary label. 1 for cafe, 0 for kopitiam
reviews_train_df['label'] = np.where(reviews_train_df['is_cafe'] == 1, 1, 0)

# drop unnecessary columns and missing reviews  
reviews_train_df = reviews_train_df.drop(columns={'place_id', 'name', 'is_cafe', 'is_kopitiam'}).dropna()
reviews_train_df.head()

,text,label
0,Junyi fortune fish. 吉祥鱼\nDabao the salted veg ...,0
1,Five star for Fish So Nice shop selling grille...,0
2,Mixed Rice stall- i don’t even know how they c...,0
3,Crazy drink lady charged me $1 for this cup of...,0
4,Dim sum stall needs improvement.\nManagement s...,0


In [172]:
reviews_train_df['label'].value_counts(normalize=True)

label
0    0.705687
1    0.294313
Name: proportion, dtype: float64

A clean labelled dataset has been prepared to train the text classification model. Due to the slight imbalanced nature of the labels, care will need to be taken to stratify the train test split accordingly 

In [174]:
# reviews_train_df.to_csv('data/interim/reviews_train.csv', index=False)